In [3]:
!pip -q install boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.40.26 which is incompatible.


In [4]:
import requests
import pandas as pd
from io import StringIO
import boto3
import os

# Cargar credenciales desde ~/.aws/credentials o variables de entorno
#session = boto3.Session()
#s3 = session.client('s3')

In [5]:
dataframes=[]

for anho in range(2014,2023):
    dict_parametros={
        "start_date":f"{anho}-09-07T00:00",
        "end_date":f"{anho +1}-09-07T23:59",
        "time_trunc":"day",
        "geo_trunc":"electric_system",
        "geo_limit":"peninsular",
        "geo_ids":"8741",
    }
    url = f"https://apidatos.ree.es/es/datos/demanda/ire-general"

    response=requests.get(url,params=dict_parametros)
    print(response.status_code)
    datos=response.json()
    datos=datos["included"][0]["attributes"]["values"]
    df=pd.DataFrame(datos)
    dataframes.append(datos)

df_master=pd.concat(dataframes,ignore_index = True)

500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df_master

In [ ]:
#con esto cuando funcione lo pondriamos en s3 guardando el dataframe en memoria

csv_buffer = StringIO()
df_master.to_csv(csv_buffer, index=False)

# Subir a S3 desde memoria
object_key = 'in_memory/productos.csv'

s3.put_object(Bucket=bucket_name, Key=object_key, Body=csv_buffer.getvalue())
print(f"DataFrame subido a S3 como '{object_key}'")

In [ ]:
#ahora leeriamos el csv para trabajar con el y ponerlo en rds
csv_key = 'in_memory/productos.csv'
s3_path_csv = f"s3://{bucket_name}/{csv_key}"

df_csv = pd.read_csv(s3_path_csv)
df_csv

In [ ]:
#nos conectamos a nuestro rds

def obtener_endpoint(id_instancia):
    try:
        db = rds.describe_db_instances(DBInstanceIdentifier=id_instancia)['DBInstances'][0]
        endpoint = db['Endpoint']['Address']
        puerto = db['Endpoint']['Port']
        print(f"🌐 Endpoint: {endpoint}:{puerto}")
        return endpoint, puerto
    except ClientError as e:
        print(f"Error: {e}")
        return None, None





In [ ]:
#creamos la base de datos demanda_eletrica y la tabla demanda usando csv de antes

def crear_base_y_tabla(endpoint, puerto):
    try:
        conn = psycopg2.connect(
            dbname='postgres',
            user='adminuser',
            password='Admin123456!',
            host=endpoint,
            port=puerto
        )
        conn.autocommit = True
        cur = conn.cursor()

        # Crear base de datos
        cur.execute("CREATE DATABASE demanda_electrica;")
        print("✅ Base de datos 'demanda_electrica' creada")

        # Conectarse a la nueva base de datos
        cur.close()
        conn.close()
        conn2 = psycopg2.connect(
            dbname='demanda_electrica',
            user='adminuser',
            password='Admin123456!',
            host=endpoint,
            port=puerto
        )
        conn2.autocommit = True
        cur2 = conn2.cursor(df_csv.to_sql(name = "demanda", con = engine, if_exists = "replace", index = False) )

        # Crear tabla
        cur2.execute()
        print("✅ Tabla 'demanda' creada en 'demanda_electrica_db'")
        cur2.close()
        conn2.close()
    except Exception as e:
        print(f"❌ Error de conexión/creación: {e}")

# 

In [ ]:
endpoint, puerto = obtener_endpoint('clase-postgres-demo')
crear_base_y_tabla(endpoint, puerto)